# Chapter 4 - Training Models

Palabras para este capítulo:
tweaks - ajustes
several - varias/varios
still - aún
switching  - transpuesta
setting - configuración

## Setup

In [1]:
import sys

assert sys.version_info >= (3, 7)

In [2]:
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1")

In [3]:
import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)

In [4]:
# Creamos un folder para guardar las imágenes generadas (si es que no existe ya), y definimos la función save_fig() 
# que se utilizará para guardar las imágenes
from pathlib import Path

IMAGES_PATH = Path() / "images" / "training_linear_models"
IMAGES_PATH.mkdir(parents=True, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

## Regresión Lineal

Estudiaremos la regresión lineal. Uno de los algoritmos más simples. Se abordará de dos métodos de entrenamientos muy distintos:
- Utilizando una ecuación cerrada (closet-form equation) la cual calcula los parámetros del modelo que más se ajustan a los datos de entrenamiento (es decir, los parámetros del modelo que minimizan la función de costo sobre el conjunto de entrenamiento)
- Usando el Descenso del Gradiente (Gradient Descent "GD") el cual ajusta gradualmente los parámetros del modelo para minimizar la función de costo sobre el conjunto de entrenamiento. Eventualmente este método converge al mismo conjunto de parámetros que el primer método.

Después revisaremos la Regresión Polinómica, un modelo más complejo que puede abordar datos no lineales. Ya que este modelo tiene más parámetros que el modelo de Regresión Lineal, es más propenso a sobre ajustarse a los datos de entrenamiento. Por lo que estaremos aprendiendo a detectar si este es el caso o no, usando curvas de aprendizaje (learning curves), y después veremos varias técnicas de regularización que pueden reducir el riezgo de sobre ajuste en los datos de entrenamiento.

De manera general, un modelo lineal hace una predicción calculando la suma ponderada de las características de entreda, mas una constante llamada término de sesgo (o término de intersección)

$$\hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + ... + \theta_n x_n$$

- $\hat{y}$ es el valor predicho.

- $n$ es el número de características.

- $x_i$ es el valor de la característica $i^{th}$.

- $\theta_j$ es el parémetro del modelo  $j^{th}$ (incluido el término de sesgo $\theta_0$ y la característica de pesos $\theta_1 + \theta_2 + ,..., + \theta_n$)

Esto puede escribirse de una forma más concisa usando la notación vectorial:



$$
\hat{y} = h_{\theta}(x) = \mathbf{\theta} \cdot \mathbf{x}
$$
(no se aprecia bien, pero $\theta$ está en negrita porque es un vector, igual que $\mathbf{x}$)

En esta ecuación:
- $\mathbf{\theta}$ es el vector de parámetros del modelo, que contiene el término de sesgo $\theta_0$ y los pesos de las características $\theta_1$ a $\theta_n$.
- $\mathbf{x}$ es el vector de características de la instancia, que contiene $x_0$ a $x_n$, con $x_0$ siempre igual a 1.
- $\mathbf{\theta} \cdot \mathbf{x}$ es el producto punto de los vectores $\theta$ y $\mathbf{x}$, que por supuesto es igual a $\theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + \cdots + \theta_n x_n$.
- $h_{\mathbf{\theta}}$ es la función de hipótesis, usando los parámetros del modelo $\theta$.

En machine learning, una función de hipótesis es una función matemática que representa el modelo que estás utilizando para hacer predicciones, es fundamental porque dedine cómo el modelo transforma las entradas en predicciones. En otras palabras, la función de hipótesis $ h_{\theta}(x)$ utiliza los parámetros $\theta$ y los valores de las características $\mathbf{x}$ para calcular la predicción de la salida.


OK, ese es el modelo de Regresión Lineal, pero ¿cómo lo entrenamos? Bueno, recordemos que entrenar un modelo significa ajustar sus parámetros para que el modelo se ajuste mejor al conjunto de entrenamiento. Para este propósito, primero necesitamos una medida de qué tan bien (o mal) el modelo se ajusta a los datos de entrenamiento. En el Capítulo 2 vimos que la medida de rendimiento más común de un modelo de regresión es el Error Cuadrático Medio de la raíz $^{\textbf{[1]}}$ (Root Mean Square Error: RMSE). Por lo tanto, para entrenar un modelo de Regresión Lineal, necesitamos encontrar el valor de $\theta$ que minimice el RMSE. En la práctica, es más sencillo minimizar el error cuadrático medio (MSE) que el RMSE, y conduce al mismo resultado (porque el valor que minimiza una función también minimiza su raíz cuadrada).

El MSE de una hipótesis de Regresión Lineal $ h_\theta(x) $ en un conjunto de entrenamiento $\mathbf{X}$ se calcula usando la siguiente ecuación.

$$
\text{MSE}(X, h_{\theta}) = \frac{1}{m} \sum_{i=1}^{m} (\theta^\top x^{(i)} - y^{(i)})^2
$$


### La Ecuación Normal
Para encontrar el valor de $ \theta $ que minimiza la función de costo, existe una solución de forma cerrada, en otras palabras, una ecuación matemática que da el resultado directamente. Esto se llama la Ecuación Normal:

$$
\hat{\theta} = (X^\top X)^{-1} X^\top y
$$

En esta ecuación:
- $ \hat{\theta} $ es el valor de $ \theta $ que minimiza la función de costo.
- $ y $ es el vector de valores objetivo que contiene $ y_1 $ a $ y_n $.

Generemos algunos datos con apariencia lineal para probar esta ecuación:


## Notas

$\textbf{[1]}$ 

En machine learning, el Root Mean Square Error (RMSE) se traduce al español como Error Cuadrático Medio de la Raíz. Es una métrica que se utiliza comúnmente para evaluar la precisión de un modelo de regresión. Se calcula como la raíz cuadrada de la media de los errores al cuadrado entre los valores predichos por el modelo y los valores reales del conjunto de datos.

En términos matemáticos, el RMSE se define como:

$$ \text{RMSE} = \sqrt{\frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y}_i)^2} $$

donde:
- $ m $ es el número de ejemplos en el conjunto de datos.
- $ y_i $ son los valores reales del objetivo para el i-ésimo ejemplo.
- $ \hat{y}_i $ son los valores predichos por el modelo para el i-ésimo ejemplo.



La ecuación del Mean Squared Error (MSE) para evaluar un modelo de regresión es simplemente:

$$ \text{MSE} = \frac{1}{m} \sum_{i=1}^{m} (y_i - \hat{y}_i)^2 $$


El MSE calcula el promedio de los cuadrados de los errores entre las predicciones del modelo ($ \hat{y}_i $) y los valores reales ($ y_i $). Es una medida de la calidad de las predicciones del modelo, donde valores más bajos indican un mejor ajuste del modelo a los datos observados.



La principal diferencia entre RMSE y Mean Squared Error (MSE) radica en la interpretación de la métrica resultante:
- **MSE**: Es simplemente la media de los errores al cuadrado entre los valores predichos y los valores reales, sin la raíz cuadrada. Por lo tanto, MSE es más sensible a valores atípicos (outliers) en los datos, ya que los errores más grandes contribuyen de manera proporcionalmente mayor a la métrica.
- **RMSE**: Al tomar la raíz cuadrada del MSE, el RMSE proporciona una medida de error que está en la misma unidad que la variable objetivo original. Esto hace que sea más interpretable, ya que representa el error promedio en la misma escala que los datos originales, además penaliza de manera más significativa los errores grandes en comparación con el MSE.

